In [1]:
import sys
import pickle as pkl
import array
import os
import timeit 
import contextlib
import numpy as np
import math 

In [2]:
with open('dataset/A Drinking Song.txt', 'r') as f:
    print(f.read())
   

Author: William Butler Yeats
Wine comes in at the mouth
And love comes in at the eye;
That's all we shall know for truth
Before we grow old and die.
I lift the glass to my mouth,
I look at you, and I sigh.


In [1]:
#定义IDMAP，实现id和str的转换
class IdMap:
    """Helper class to store a mapping from strings to ids."""
    def __init__(self):
        self.str_to_id = {}
        self.id_to_str = []
        
    def __len__(self):
        """Return number of terms stored in the IdMap"""
        return len(self.id_to_str)
        
    def _get_str(self, i):
        """Returns the string corresponding to a given id (`i`)."""
        ### Begin your code
        return self.id_to_str[i]
        ### End your code
        
    def _get_id(self, s):
        """Returns the id corresponding to a string (`s`). 
        If `s` is not in the IdMap yet, then assigns a new id and returns the new id.
        """
        ### Begin your code
        if s not in self.str_to_id:
            self.id_to_str.append(s)
            temp=self.id_to_str.index(s)
            self.str_to_id[s]=temp
            #返回新分配的id
            return temp
        
        #如果s在，返回已存在的id
        return self.str_to_id[s]
        ### End your code
            
    def __getitem__(self, key):
        """If `key` is a integer, use _get_str; 
           If `key` is a string, use _get_id;"""
        if type(key) is int:
            return self._get_str(key)
        elif type(key) is str:
            return self._get_id(key)
        else:
            raise TypeError

## 一、数据集处理、建立向量空间

### （1）词袋生成、map构建
如下为遍历数据集，生成标题、作者和内容的词袋。

In [4]:
#分别为标题、作者和内容的词袋
bag_words_title=[]
bag_words_author=[]
bag_words_content=[]

#数据集地址
dir="dataset/"
#文档的idmap
doc_id_map=IdMap()
#统计文档数
num_doc=0

for doc in os.listdir(dir):
    num_doc+=1
    #去除.txt
    pre_doc=doc.strip()[0:-4].lower()
    for term_title in pre_doc.split(' '):
        #题目构成的词袋
        bag_words_title.append(term_title)
        
    #添加doc和id的map
    doc_id=doc_id_map[doc.strip()[0:-4]]

    firstline=1
    for line in open(dir+doc).readlines(): 
        if firstline==1:
            line=line.strip()[8:].lower().split(' ')       
            bag_words_author.extend(line)
            firstline=0
        else:
            #去除停用词
            line=line.replace(';','')
            line=line.replace(",", '')
            line=line.replace('.','')
            line=line.replace(':','')
            line=line.replace('?','')
            line=line.replace('!','')
            line=line.replace("'s",'')
            line=line.replace("'ll",'')
            line=line.replace("'",'')  
            line=line.strip().lower().split(' ')  
            bag_words_content.extend(line)

            
            
bag_words_title=sorted(set(bag_words_title))
bag_words_author=sorted(set(bag_words_author))
bag_words_content=sorted(set(bag_words_content))
print("bag_words_title: ",bag_words_title)
print("bag_words_author:",bag_words_author)
print("bag_words_content: ",bag_words_content)

bag_words_title:  ['a', 'aedh', 'are', 'by', 'can', 'cloths', 'down', 'drinking', 'fear', 'for', 'freedom', 'gardens', 'heaven', 'i', 'in', 'is', 'leave', 'me', 'mind', 'moonlight', 'of', 'old', 'salley', 'song', 'the', 'this', 'tonight', 'walk', 'when', 'where', 'wishes', 'with', 'without', 'write', 'you']
bag_words_author: ['butler', 'knight', 'leon', 'neruda', 'pablo', 'rabindranath', 'tagore', 'william', 'yeats']
bag_words_content:  ['a', 'adventurous', 'again', 'ages', 'agree', 'all', 'am', 'anarchy', 'and', 'are', 'arms', 'as', 'at', 'back', 'beads', 'beauty', 'because', 'beckoning', 'been', 'before', 'being', 'bending', 'bid', 'blind', 'blinding', 'blue', 'book', 'breaking', 'broken', 'burden', 'but', 'by', 'call', 'can', 'changing', 'chanting', 'charms', 'claim', 'clear', 'close', 'closer', 'cloths', 'cold', 'come', 'comes', 'corner', 'cup', 'dark', 'day', 'dead', 'death', 'deep', 'depth', 'desert', 'destiny', 'did', 'die', 'dim', 'distance', 'domestic', 'doors', 'dost', 'down'

根据生成标题、作者和内容的词袋构建IDMAP

In [5]:
#分别构建map
print(len(bag_words_title))
print(len(bag_words_author))
print(len(bag_words_content))

title_term_id_map=IdMap()
author_term_id_map=IdMap()
content_term_id_map=IdMap()

for temp in bag_words_title:
    t=title_term_id_map[temp]
    
for temp in bag_words_author:
    t=author_term_id_map[temp]
    
for temp in bag_words_content:
    t=content_term_id_map[temp]

print(title_term_id_map.str_to_id)
print(author_term_id_map.str_to_id)
print(content_term_id_map.str_to_id)


35
9
277
{'a': 0, 'aedh': 1, 'are': 2, 'by': 3, 'can': 4, 'cloths': 5, 'down': 6, 'drinking': 7, 'fear': 8, 'for': 9, 'freedom': 10, 'gardens': 11, 'heaven': 12, 'i': 13, 'in': 14, 'is': 15, 'leave': 16, 'me': 17, 'mind': 18, 'moonlight': 19, 'of': 20, 'old': 21, 'salley': 22, 'song': 23, 'the': 24, 'this': 25, 'tonight': 26, 'walk': 27, 'when': 28, 'where': 29, 'wishes': 30, 'with': 31, 'without': 32, 'write': 33, 'you': 34}
{'butler': 0, 'knight': 1, 'leon': 2, 'neruda': 3, 'pablo': 4, 'rabindranath': 5, 'tagore': 6, 'william': 7, 'yeats': 8}
{'a': 0, 'adventurous': 1, 'again': 2, 'ages': 3, 'agree': 4, 'all': 5, 'am': 6, 'anarchy': 7, 'and': 8, 'are': 9, 'arms': 10, 'as': 11, 'at': 12, 'back': 13, 'beads': 14, 'beauty': 15, 'because': 16, 'beckoning': 17, 'been': 18, 'before': 19, 'being': 20, 'bending': 21, 'bid': 22, 'blind': 23, 'blinding': 24, 'blue': 25, 'book': 26, 'breaking': 27, 'broken': 28, 'burden': 29, 'but': 30, 'by': 31, 'call': 32, 'can': 33, 'changing': 34, 'chanting

### （2）生成tf

In [6]:
#tf向量
tf_title=np.zeros((len(bag_words_title),num_doc))
tf_author=np.zeros((len(bag_words_author),num_doc))
tf_content=np.zeros((len(bag_words_content),num_doc))
print(tf_author.shape)


'''
之后的计算中TF = log10( N+1)，减少文本长度带来的应影响

向量形如：
term  doc1    doc2
a     0       2
b     1       2
c     0       1
'''

#统计文档数
doc_count=0

#遍历数据集
for doc in os.listdir(dir):
    #将得到的标题进行预处理
    pre_doc=doc.strip()[0:-4].lower()
    #根据预处理后的标题构建tf向量
    for term_title in pre_doc.split(' '):
        term_id=title_term_id_map._get_id(term_title)
        tf_title[term_id][doc_count]+=1
    

    firstline=1
    for line in open(dir+doc).readlines(): 
        #判断是否为第一行，第一行为作者
        if firstline==1:
            #作者信息预处理
            line=line.strip()[8:].lower().split(' ')
            #构建作者的tf向量
            for term_author in line:
                author_id=author_term_id_map._get_id(term_author)
                tf_author[author_id][doc_count]+=1
            firstline=0
        #内容   
        else:
            #去除停用词
            line=line.replace(';','')
            line=line.replace(",", '')
            line=line.replace('.','')
            line=line.replace(':','')
            line=line.replace('?','')
            line=line.replace('!','')
            line=line.replace("'s",'')
            line=line.replace("'ll",'')
            line=line.replace("'",'')
            #内容预处理
            line=line.strip().lower().split(' ')
            #构建内容的tf向量
            for term_content in line:
                content_id=content_term_id_map._get_id(term_content)
                tf_content[content_id][doc_count]+=1
            
    doc_count+=1

(9, 9)


'\n之后的计算中TF = log10( N+1)，减少文本长度带来的应影响\n\n向量形如：\nterm  doc1    doc2\na     0       2\nb     1       2\nc     0       1\n'

In [7]:
print(author_term_id_map.str_to_id)
print(tf_author)
#print(title_term_id_map.str_to_id)
#print(tf_title)
#print(content_term_id_map.str_to_id)
#print(tf_content)

{'butler': 0, 'knight': 1, 'leon': 2, 'neruda': 3, 'pablo': 4, 'rabindranath': 5, 'tagore': 6, 'william': 7, 'yeats': 8}
[[1. 1. 1. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 1.]
 [0. 0. 0. 1. 1. 0. 0. 0. 1.]
 [1. 1. 1. 0. 0. 0. 0. 1. 0.]
 [1. 1. 1. 0. 0. 0. 0. 1. 0.]]


### （3）生成df、idf



In [8]:
#df和idf向量

df_title=np.zeros((len(bag_words_title),2))
df_author=np.zeros((len(bag_words_author),2))
df_content=np.zeros((len(bag_words_content),2))

'''
其中IDF = log10( N/df)，其中N为文档数,df为含有对应term的文档数

向量形如：
term  df  idf
a    2  log10(N/2)
b    1  log10(N/1)
c    1  log10(N/1)
'''

#生成term对应的df和idf
def df(len,num_doc,tf_x,df_x):
    for i in range(len):
        count=0
        for j in range(num_doc):
            if tf_x[i][j]!=0:
                count+=1
        df_x[i][0]=count
        df_x[i][1]=math.log10(num_doc/count)


df(len(bag_words_title),num_doc,tf_title,df_title)
df(len(bag_words_author),num_doc,tf_author,df_author)
df(len(bag_words_content),num_doc,tf_content,df_content)
    


print(df_author)

'\n其中IDF = log10( N/df)，其中N为文档数,df为含有对应term的文档数\n\n向量形如：\nterm  df  idf\na    2  log10(N/2)\nb    1  log10(N/1)\nc    1  log10(N/1)\n'

[[4.         0.35218252]
 [1.         0.95424251]
 [1.         0.95424251]
 [1.         0.95424251]
 [1.         0.95424251]
 [3.         0.47712125]
 [3.         0.47712125]
 [4.         0.35218252]
 [4.         0.35218252]]


### （4）tf-idf
分别生成文档的标题、作者、内容的tf-idf向量

In [9]:
#tf-idf


tf_idf_title=np.zeros((num_doc,len(bag_words_title)))
tf_idf_author=np.zeros((num_doc,len(bag_words_author)))
tf_idf_content=np.zeros((num_doc,len(bag_words_content)))

#生成文档对对应的tf-idf向量
def tf_idf(num_doc,len,tf_idf_x,tf_x,df_x):
    
    for i in range(num_doc):
        for j in range(len):
            tf_idf_x[i][j]=math.log10(tf_x[j][i]+1)*df_x[j][1]
            
            
tf_idf(num_doc,len(bag_words_title),tf_idf_title,tf_title,df_title)
tf_idf(num_doc,len(bag_words_author),tf_idf_author,tf_author,df_author)
tf_idf(num_doc,len(bag_words_content),tf_idf_content,tf_content,df_content)
print(tf_idf_author)


[[0.1060175  0.         0.         0.         0.         0.
  0.         0.1060175  0.1060175 ]
 [0.1060175  0.         0.         0.         0.         0.
  0.         0.1060175  0.1060175 ]
 [0.1060175  0.         0.         0.         0.         0.
  0.         0.1060175  0.1060175 ]
 [0.         0.         0.         0.         0.         0.14362781
  0.14362781 0.         0.        ]
 [0.         0.         0.         0.         0.         0.14362781
  0.14362781 0.         0.        ]
 [0.         0.         0.         0.28725562 0.28725562 0.
  0.         0.         0.        ]
 [0.         0.28725562 0.28725562 0.         0.         0.
  0.         0.         0.        ]
 [0.1060175  0.         0.         0.         0.         0.
  0.         0.1060175  0.1060175 ]
 [0.         0.         0.         0.         0.         0.14362781
  0.14362781 0.         0.        ]]


### （5）序列化储存

In [10]:
try:
    os.mkdir("pkl_dir")
except FileExistsError:
    pass

with open('pkl_dir/doc.pkl','wb') as doc:
    pkl.dump((num_doc,doc_id_map),doc)


with open('pkl_dir/term_map.pkl','wb') as tp:
    pkl.dump((title_term_id_map,author_term_id_map,content_term_id_map),tp)

with open('pkl_dir/bag_words.pkl','wb') as bw:
    pkl.dump((bag_words_title,bag_words_author,bag_words_content),bw)

with open('pkl_dir/tf.pkl','wb') as tf:
    pkl.dump((tf_title,tf_author,tf_content),tf)
    
with open('pkl_dir/df.pkl','wb') as df:
    pkl.dump((df_title,df_author,df_content),df)
    
with open('pkl_dir/tf_idf.pkl','wb') as tf_idf :
    pkl.dump((tf_idf_title,tf_idf_author,tf_idf_content),tf_idf)





以上将数据集处理完毕并存储

## 二、查询
### （1）读取存储的向量及相关信息

In [3]:

import sys
import pickle as pkl
import array
import os
import timeit 
import contextlib
import numpy as np
import math 

#定义IDMAP，实现id和str的转换
class IdMap:
    """Helper class to store a mapping from strings to ids."""
    def __init__(self):
        self.str_to_id = {}
        self.id_to_str = []
        
    def __len__(self):
        """Return number of terms stored in the IdMap"""
        return len(self.id_to_str)
        
    def _get_str(self, i):
        """Returns the string corresponding to a given id (`i`)."""
        ### Begin your code
        return self.id_to_str[i]
        ### End your code
        
    def _get_id(self, s):
        """Returns the id corresponding to a string (`s`). 
        If `s` is not in the IdMap yet, then assigns a new id and returns the new id.
        """
        ### Begin your code
        if s not in self.str_to_id:
            self.id_to_str.append(s)
            temp=self.id_to_str.index(s)
            self.str_to_id[s]=temp
            #返回新分配的id
            return temp
        
        #如果s在，返回已存在的id
        return self.str_to_id[s]
        ### End your code
            
    def __getitem__(self, key):
        """If `key` is a integer, use _get_str; 
           If `key` is a string, use _get_id;"""
        if type(key) is int:
            return self._get_str(key)
        elif type(key) is str:
            return self._get_id(key)
        else:
            raise TypeError

反序列化

In [4]:

bag_words_title=[]
bag_words_author=[]
bag_words_content=[]
num_doc=0
doc_id_map=IdMap()

with open('pkl_dir/doc.pkl','rb') as doc:
    num_doc,doc_id_map= pkl.load(doc)

with open('pkl_dir/bag_words.pkl','rb') as bw:
    bag_words_title,bag_words_author,bag_words_content= pkl.load(bw)

tf_title=np.zeros((len(bag_words_title),num_doc))
tf_author=np.zeros((len(bag_words_author),num_doc))
tf_content=np.zeros((len(bag_words_content),num_doc))
    
title_term_id_map=IdMap()
author_term_id_map=IdMap()
content_term_id_map=IdMap()  

with open('pkl_dir/term_map.pkl','rb') as tp:
    title_term_id_map,author_term_id_map,content_term_id_map=pkl.load(tp)

with open('pkl_dir/tf.pkl','rb') as tf:
    tf_title,tf_author,tf_content=pkl.load(tf)
    
df_title=np.zeros((len(bag_words_title),2))
df_author=np.zeros((len(bag_words_author),2))
df_content=np.zeros((len(bag_words_content),2))
    
with open('pkl_dir/df.pkl','rb') as df:
    df_title,df_author,df_content=pkl.load(df)
    
tf_idf_title=np.zeros((num_doc,len(bag_words_title)))
tf_idf_author=np.zeros((num_doc,len(bag_words_author)))
tf_idf_content=np.zeros((num_doc,len(bag_words_content)))
    
with open('pkl_dir/tf_idf.pkl','rb') as tf_idf :
    tf_idf_title,tf_idf_author,tf_idf_content=pkl.load(tf_idf)


### (2) 输入的处理、生成检索向量的函数

In [5]:
#构建输入检索的向量

#输入的预处理
def pre_input(line):
         #去除停用词
        line=line.replace(';','')
        line=line.replace(",", '')
        line=line.replace('.','')
        line=line.replace(':','')
        line=line.replace('?','')
        line=line.replace('!','')
        line=line.replace("'s",'')
        line=line.replace("'ll",'')
        line=line.replace("'",'')
        #内容预处理
        line=line.strip().lower().split(' ')
        
        return line

def query_vector(input_query,len,bag_words_x,x_term_id_map,df_x,x_query_vector):
    bag_words=pre_input(input_query)
    
    tf=np.zeros(len)
    for term in bag_words:
        if term in bag_words_x:
            term_id=x_term_id_map._get_id(term)
            tf[term_id]+=1

    
    for i in range(len):
        x_query_vector[i]=math.log10(tf[i]+1)*df_x[i][1]
    
    return tf_idf




### (3) 查询输入、根据查询生成查询向量

In [22]:
#输入
title_query=input("please input the title you want to search: ")
author_query=input("please input the author you want to search: ")
content_query=input("please input the content you want to search: ")

#生成输入的检索的向量
title_query_vector=np.zeros(len(bag_words_title))
author_query_vector=np.zeros(len(bag_words_author))
content_query_vector=np.zeros(len(bag_words_content))
                            
#独热码，有相应查询，设为1，生成对应查询向量                           
query=[0,0,0]
if title_query!='':
    query[0]=1
    query_vector(title_query,len(bag_words_title),bag_words_title,title_term_id_map,df_title,title_query_vector)
if author_query!='':
    query[1]=1
    query_vector(author_query,len(bag_words_author),bag_words_author,author_term_id_map,df_author,author_query_vector)
if content_query!='':
    query[2]=1
    query_vector(content_query,len(bag_words_content),bag_words_content,content_term_id_map,df_content,content_query_vector)


please input the title you want to search: 
please input the author you want to search: 
please input the content you want to search: I would spread the cloths under your feet:Drink deep the cup of moonlight; Drink deep the magic charms.


<_io.BufferedReader name='pkl_dir/tf_idf.pkl'>

### （4）计算得分，排序，输出

In [23]:
#各个文档分数
score_title=np.zeros(num_doc)
score_author=np.zeros(num_doc)
score_content=np.zeros(num_doc)

#分配权重
w_title=5
w_author=3
w_content=2
w_sum=0


#cos
def cosine_similarity(x,y):
    num = x.dot(y.T)
    denom = np.linalg.norm(x) * np.linalg.norm(y)
    return num / denom

#分数计算
def get_score(len,score,tf_idf_x,query):
    
    for i in range(len):
        score[i]=cosine_similarity(tf_idf_x[i],query)

#根据独热码，计算单个域的得分向量，计算对应的权重和
if query[0]==1:
    get_score(num_doc,score_title,tf_idf_title,title_query_vector)
    w_sum+=w_title
if query[1]==1:
    get_score(num_doc,score_author,tf_idf_author,author_query_vector)
    w_sum+=w_author
if query[2]==1:
    get_score(num_doc,score_content,tf_idf_content,content_query_vector)
    w_sum+=w_content
            



#文档分数向量

score=np.zeros(num_doc)
for i in range(num_doc):
    if query[0]==1:
         score[i]+=(w_title/w_sum)*score_title[i]
    if query[1]==1:
        score[i]+=(w_author/w_sum)*score_author[i]
    if query[2]==1:
        score[i]+=(w_content/w_sum)*score_content[i]
            

    



#根据相关性得分排序，去除得分为0文档，其他文档按从大到小排序
sorted_score = sorted(enumerate(score), key=lambda score:score[1],reverse=True)  
sorted_score_id = [score[0] for score in sorted_score if score[1]!=0] 


#根据排名输出
for i in sorted_score_id:
    str=doc_id_map._get_str(i)
    print(str)
    
    


Walk with Me in Moonlight
Aedh wishes for the Cloths of Heaven
When you are old
Tonight I Can Write
Down by the Salley Gardens
Freedom
A Drinking Song
Where the Mind is Without Fear
Leave This
